In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Data Mining/dataset_mood_smartphone.csv')


In [ ]:
df.head()

,Unnamed: 0,id,time,variable,value
0,1,AS14.01,2014-02-26 13:00:00.000,mood,6.0
1,2,AS14.01,2014-02-26 15:00:00.000,mood,6.0
2,3,AS14.01,2014-02-26 18:00:00.000,mood,6.0
3,4,AS14.01,2014-02-26 21:00:00.000,mood,7.0
4,5,AS14.01,2014-02-27 09:00:00.000,mood,6.0


In [ ]:
df.dtypes

Unnamed: 0      int64
id             object
time           object
variable       object
value         float64
dtype: object

The datatypes

In [ ]:
df.isna().sum()

Unnamed: 0      0
id              0
time            0
variable        0
value         202
dtype: int64

- 202 missing values in the column "values"

In [ ]:
df.describe(include="all")

In [ ]:
x = df['id'].value_counts()
cols = [i for i in x.axes[0]]
vals = [i for i in x]
plt.bar(cols, vals, width = 0.4)
 
plt.xlabel("ID")
plt.ylabel("Count")
plt.xticks(rotation=90)
plt.show()

Error in callback <function _draw_all_if_interactive at 0x7f686c282700> (for post_execute):


KeyboardInterrupt: ignored